# soundNet: Classifying Vehicle Classes from Audio Files

Import Libraries/Modules

config

In [ ]:
# Configuration cell
# ------------------
# This cell contains all the configuration for the notebook.

# Path to the directory containing the sound files.
audio_dir = '../data/audio'
annotations_dir = '../data/annotations'

## Data Manipulation

### Loading audio files